In [1]:
import os
from pathlib import Path
import glob
import json
import random
import shutil
from utils import wavwrite, read_scaled_wav, fix_length
from random import randrange

tat_dir = "D:/DP_files/TAT-2mix/min/"

tai_dir = "D:/DP_files/condenser/"
noise_dir = "D:/DP_files/wham_noise/"

# Create output directories
for mode in ["tr", "cv", "tt"]:
    os.makedirs(tat_dir + mode + "/mix_clean")
    os.makedirs(tat_dir + mode + "/mix_both")
    os.makedirs(tat_dir + mode + "/s1")
    os.makedirs(tat_dir + mode + "/s2")

# Collect filenames from input directories
tai_list = [w for w in Path(tai_dir).glob('**/*.wav') if os.path.isfile(w)]
tai_list.sort()
noise_list = [w for w in Path(noise_dir).glob('**/*.wav') if os.path.isfile(w)]
noise_list.sort()

# Combine speakers and save files to output directory
all_counter = 0
for i in range(len(tai_list)):
    if all_counter == 28000:
            break
    for j in range(len(tai_list)):
        if i == j:
            continue
        if all_counter == 28000:
            break
        # Load audio data from input files
        audio1 = read_scaled_wav(tai_list[i], 1, 8000)
        audio2 = read_scaled_wav(tai_list[j], 1, 8000)
        noise = read_scaled_wav(noise_list[all_counter], 1, 8000)

        # Mix audio and save to output file
        audio1, audio2 = fix_length(audio1, audio2, "tat") 
        # přepsal jsem si fix_length "eat" ať to bere konec nahrávky, protože v tat mluví nakonci  a na začátku je ticho
        if(len(noise) > len(audio1)):
            audio1, noise = fix_length(audio1, noise, "min")
        else:
            audio1, noise = fix_length(audio1, noise, "max")

        mixed_speech = audio1 + audio2
        mixed_noisy = audio1 + audio2 + noise
        if all_counter < 20000:
            output_file = f"{all_counter}.wav"
            wavwrite(tat_dir + "tr/mix_clean/" + output_file, mixed_speech, 8000)
            wavwrite(tat_dir + "tr/mix_both/" + output_file, mixed_noisy, 8000)
            wavwrite(tat_dir + "tr/s1/" + output_file, audio1, 8000)
            wavwrite(tat_dir + "tr/s2/" + output_file, audio2, 8000)
        elif all_counter < 25000:
            output_file = f"{all_counter-19999}.wav"
            wavwrite(tat_dir + "cv/mix_clean/" + output_file, mixed_speech, 8000)
            wavwrite(tat_dir + "cv/mix_both/" + output_file, mixed_noisy, 8000)
            wavwrite(tat_dir + "cv/s1/" + output_file, audio1, 8000)
            wavwrite(tat_dir + "cv/s2/" + output_file, audio2, 8000)
        elif all_counter < 28000:
            output_file = f"{all_counter-24999}.wav"
            wavwrite(tat_dir + "tt/mix_clean/" + output_file, mixed_speech, 8000)
            wavwrite(tat_dir + "tt/mix_both/" + output_file, mixed_noisy, 8000)
            wavwrite(tat_dir + "tt/s1/" + output_file, audio1, 8000)
            wavwrite(tat_dir + "tt/s2/" + output_file, audio2, 8000)
        all_counter += 1